In [97]:
import discogs_client

In [98]:
def readUserToken():
    try:
        with open("user_token.txt", "r") as file:
            content = file.read()
            return content
    except FileNotFoundError:
        return {}

d = discogs_client.Client('dardok_app/0.1', user_token='{}'.format(readUserToken().rstrip()))

In [99]:
label_id = 2716454
min_average = 4.8

In [100]:
result_label = d.label(label_id)

In [101]:
def getFilteredReleaseListFromLabel(label):
    """" A function that returns all the filtered releases from label """
    filtered_release_list = []
    for k in range(1, result_label.releases.pages+1):
        print()
        print("Page: {}/{}".format(k,result_label.releases.pages))
        print(result_label.releases.page(k))
        for item in result_label.releases.page(k):
            release = d.release(item.id)       
            if release.community.rating.average >= min_average and release.formats[0]['name']=='Vinyl':
                filtered_release_list.append(release)
    return filtered_release_list

filtered_release_list = getFilteredReleaseListFromLabel(result_label)


Page: 1/1
[<Release 22873727 'Pleistocene Future 1'>, <Release 25948969 'Pleistocene Future 2'>, <Release 26702471 'Pleistocene Future 3'>, <Release 28200046 'Pleistocene Future 4'>, <Release 29127436 'Pleistocene Future 5'>]


In [102]:
def funcYear(e):
    """" A function that returns the 'year' value """
    return e.year

filtered_release_list.sort(reverse=True, key=funcYear)

In [103]:
# Define function to remove duplicates from an input_list by id
def remove_duplicates(input_list):
    """ A function to remove duplicates from a release input_list by id """
    unique_list = []
    seen_ids = set()

    for item in input_list:
        item_id = item.id
        if item_id not in seen_ids:
            seen_ids.add(item_id)
            unique_list.append(item)

    return unique_list

unique_filtered_release_list = remove_duplicates(filtered_release_list)

In [104]:
def writeOutput(label, release_list):
    """ A function to write output as plain text"""
    print("FILETERED RELEASES for label {} - id: {}".format(label.name, label.id))
    print("{}".format(label.url))
    for release in release_list:
        print()
        print("Title: {}".format(release.title))
        print("Artists: {}".format(release.artists))
        print("Year: {}".format(release.year))
        print("Labels: {}".format(release.labels))
        print("Genres: {}".format(release.genres))
        print("Styles: {}".format(release.styles))
        print("Formats: {}".format(release.formats))
        print("Community rating average/count: {}/{} ".format(release.community.rating.average, release.community.rating.count))
        print("Url: {}".format(release.url))

writeOutput(result_label, unique_filtered_release_list)

FILETERED RELEASES for label Pleistocene Future - id: 2716454
https://www.discogs.com/label/2716454-Pleistocene-Future

Title: Pleistocene Future 2
Artists: [<Artist 57475 'Bas Mooy'>]
Year: 2023
Labels: [<Label 2716454 'Pleistocene Future'>]
Genres: ['Electronic']
Styles: ['Techno']
Formats: [{'name': 'Vinyl', 'qty': '1', 'descriptions': ['12"']}]
Community rating average/count: 4.83/6 
Url: https://www.discogs.com/release/25948969-Bas-Mooy-Pleistocene-Future-2

Title: Pleistocene Future 4
Artists: [<Artist 13491121 'Dario Evangelista'>]
Year: 2023
Labels: [<Label 2716454 'Pleistocene Future'>]
Genres: ['Electronic']
Styles: ['Techno']
Formats: [{'name': 'Vinyl', 'qty': '1', 'descriptions': ['12"', '33 ⅓ RPM', 'EP']}]
Community rating average/count: 4.83/6 
Url: https://www.discogs.com/release/28200046-Dario-Evangelista-Pleistocene-Future-4

Title: Pleistocene Future 5
Artists: [<Artist 5153732 'ARKVS'>]
Year: 2023
Labels: [<Label 2716454 'Pleistocene Future'>]
Genres: ['Electronic']


In [105]:
def get_artists(release):
    artists = []
    for artist in release.artists:
        artists.append(artist.name)
    return artists

def get_labels(release):
    labels = []
    for label in release.labels:
        labels.append(label.name)
    return labels

In [106]:
def writeOutputHtml(label, release_list):
    """ A function to write output as HTML """

    htmlString = "<html>\n<head>\n<title>Filtered Releases for label {} - id: {}</title>\n</head>\n \
                    <body>\n \
                        <h1>Filtered Releases for label {} - id: {}</h1>\n \
                        <h2>Url: <a href='{}' target='_blank' rel='noopener noreferrer'>{}</a><h2>\n \
                    ".format(label.name, label.id, label.name, label.id, label.url, label.url)

    htmlString += "<ul style='list-style: none;'>\n"
    for release in release_list:
        htmlString += "<br>\n"
        htmlString += "<li><span style='color: #9b9b9b'>Title:</span> {}</li>\n".format(release.title)
        artists = get_artists(release)
        htmlString += "<li><span style='color: #9b9b9b'>Artists:</span> {}</li>\n".format(artists)
        htmlString += "<li><span style='color: #9b9b9b'>Year:</span> {}</li>\n".format(release.year)
        labels = get_labels(release)
        htmlString += "<li><span style='color: #9b9b9b'>Labels:</span> {}</li>\n".format(labels)
        htmlString += "<li><span style='color: #9b9b9b'>Genres:</span> {}</li>\n".format(release.genres)
        htmlString += "<li><span style='color: #9b9b9b'>Styles:</span> {}</li>\n".format(release.styles)
        htmlString += "<li><span style='color: #9b9b9b'>Formats:</span> {}</li>\n".format(release.formats)
        htmlString += "<li><span style='color: #9b9b9b'>Community rating average/count:</span> <span style='color: #ff0000'>{}</span>/<span style='color: #ff8000'>{}</span></li>\n".format(release.community.rating.average, release.community.rating.count)
        htmlString += "<li><span style='color: #9b9b9b'>Url:</span> <a href='{}' target='_blank' rel='noopener noreferrer'>{}</a></li>\n".format(release.url, release.url)

    htmlString += "</ul>\n"
    
    htmlString += "</body>\n \
                  </html>"
    
    with open("outputHtmlFile.html", "w") as file:
        file.write(htmlString)

writeOutputHtml(result_label, unique_filtered_release_list)